In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Define paths to your training and testing directories
train_data_dir = '/content/drive/MyDrive/Car_dataset_final/Train'
test_data_dir = '/content/drive/MyDrive/Car_dataset_final/Test'

# Constants
img_width, img_height = 224, 224
num_epochs = 50
batch_size = 32
num_classes = len(os.listdir(train_data_dir))

# Data Generators with Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

# Load DenseNet121 base model
base_model = DenseNet121(weights='imagenet', include_top=False)

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Combine base model with custom layers
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze layers in base model except the last few
for layer in base_model.layers[:-10]:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[early_stopping, model_checkpoint, reduce_lr])

# Load the best model
model = tf.keras.models.load_model('best_model.h5')

# Predict function
def predict_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(img_width, img_height))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0  # Rescale to [0,1]
    prediction = model.predict(img_array)
    predicted_class_index = np.argmax(prediction)
    brand_names = sorted(os.listdir(train_data_dir))
    predicted_brand_name = brand_names[predicted_class_index]
    return predicted_brand_name

# Example usage:
image_path = '/content/drive/MyDrive/Car_dataset_final/Test/mercedes/30.jpg'  # Replace with the path to the image you want to predict
predicted_brand_name = predict_image(image_path)
print("Predicted brand name:", predicted_brand_name)


Found 64 images belonging to 3 classes.
Found 58 images belonging to 3 classes.
29084464/29084464 [==============================] - 0s 0us/step
Epoch 1/50
2/2 [==============================] - ETA: 0s - loss: 3.1313 - accuracy: 0.3125 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


2/2 [==============================] - 54s 38s/step - loss: 3.1313 - accuracy: 0.3125 - val_loss: 4.0164 - val_accuracy: 0.2812 - lr: 0.0010
Epoch 2/50
2/2 [==============================] - 19s 12s/step - loss: 3.0357 - accuracy: 0.3750 - val_loss: 2.2863 - val_accuracy: 0.1875 - lr: 0.0010
Epoch 3/50
2/2 [==============================] - 18s 12s/step - loss: 1.0206 - accuracy: 0.5469 - val_loss: 0.6634 - val_accuracy: 0.7188 - lr: 0.0010
Epoch 4/50
2/2 [==============================] - 24s 16s/step - loss: 1.1758 - accuracy: 0.5156 - val_loss: 0.5940 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 5/50
2/2 [==============================] - 19s 12s/step - loss: 0.9666 - accuracy: 0.6875 - val_loss: 0.5018 - val_accuracy: 0.7812 - lr: 0.0010
Epoch 6/50
2/2 [==============================] - 18s 12s/step - loss: 0.3784 - accuracy: 0.8438 - val_loss: 0.8193 - val_accuracy: 0.6562 - lr: 0.0010
Epoch 7/50
2/2 [==============================] - 18s 11s/step - loss: 0.3262 - accuracy: 0.8438 - 

In [4]:

# Example usage:
image_path = '/content/drive/MyDrive/Car_dataset_final/Test/audi/27.jpg'  # Replace with the path to the image you want to predict
predicted_brand_name = predict_image(image_path)
print("Predicted brand name:", predicted_brand_name)

1/1 [==============================] - 0s 160ms/step
Predicted brand name: audi
